# Кластеризация

На этом семинаре мы займёмся задачей кластеризации векторов, интерпретацией полученных результатов. В частности, посмотрим на задачу выделения отдельных видов товаров в категории объявлений как на задачу кластеризации.

Нам предстоит: 

* посмотреть на подготовку векторов к кластеризации,
* реализовать алгоритм KMeans и попробовать разные варианты его инициализации,
* изучить примеры интерпретации полученных кластеров,
* ознакомиться с подбором параметров DBSCAN,
* решить задачу разделения категории на виды товаров.

## Задача анализа заголовков в категории «Товары для школы»
Цель — проанализировать, какие товары продаются в данной категории и какие заголовки там бывают для последующего использования этой информации при решении уже какой-то конкретной задачи.

Мы имеем дело с текстовыми данными — чтобы кластеризовать их, нам необходимы вектора. Одними из самых простых методов получения векторов, пригодных для использования в реальных задачах, являются TfidfVectorizer или Word2Vec.

В этом шаге мы воспользуемся моделью Word2Vec, предполагая, что она лучше подходит для смыслового анализа слов.

Пайплайн получения векторов для заголовков из семинара:

In [1]:
!pip install gensim==4.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 4.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.0
    Uninstalling gensim-4.3.0:
      Successfully uninstalled gensim-4.3.0


In [3]:
import re
from urllib import request

import numpy as np
import pandas as pd
from gensim.models import Word2Vec

Данные можно [скачать](https://stepik.org/media/attachments/lesson/535633/titles_for_clustering_original.csv)
    или загрузить по ссылке: 

In [ ]:
data = pd.read_csv('Downloads/unsupervised_learning/titles_for_clustering_original.csv')

In [ ]:
data.head(50)

In [ ]:
data.shape

In [ ]:
data

In [ ]:
# предобработка
def preprocessing(text):
    """Делаем предобработку.
    
    - приводим к нижнему регистру
    - удаляем пунктуацию и цифры
    - удаляем английские слова: не имеют большой значимости для данной задачи
    :return: строка обработанного текста
    """
    text = re.sub(r'[^А-я]', ' ', text.lower())
    return ' '.join(text.split())

data['title'] = data['title'].apply(preprocessing)
# удаляем заголовки, которые стали пустыми после предобработки
mask = data['title'].apply(lambda x: len(x.split()) > 0)
data = data[mask]

# data.drop_duplicates('title', inplace=True) - опционально, зависит от цели кластеризации
data = data.reset_index(drop=True)

In [ ]:
data.head(50)

In [ ]:
# обучение Word2Vec
w2v_corpus = [x.split() for x in data['title']]

w2v_model = Word2Vec(min_count=0, vector_size=100, hs=1, window=1, seed=0)
w2v_model.build_vocab(w2v_corpus)
w2v_model.train(w2v_corpus, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs) 

In [ ]:
w2v_model.wv['учебник']

In [ ]:
np.linalg.norm(w2v_model.wv['учебник'] - w2v_model.wv['книга'])

In [ ]:
# получение векторов заголовков из векторов слов
w2v_embeddings = []
for title_words in w2v_corpus:
    # не забываем, что некоторые слова могут не быть в словаре Word2Vec из-за выставленного min_count
    title_word_embs = [
        w2v_model.wv[word] 
        for word in title_words 
        if word in w2v_model.wv.key_to_index
    ]
    title_emb = sum(title_word_embs) / len(title_word_embs)
    w2v_embeddings.append(title_emb)
w2v_embeddings = np.vstack(w2v_embeddings)

In [ ]:
w2v_embeddings.shape

## Визуализация кластеров


In [ ]:
import random

import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

Часто используются два базовых метода анализа получившихся кластеров:

* Посмотреть на несколько ближайших объектов до каждого центроида, исходя из этого оценить содержимое всего кластера.


In [ ]:
kmeans = KMeans(n_clusters=19)
kmeans.fit(w2v_embeddings)

top = 5
clusters = kmeans.labels_
dist_to_centers = euclidean_distances(kmeans.cluster_centers_, w2v_embeddings)
top_most_similar = np.argsort(dist_to_centers)[:, :top]

for clt in np.unique(clusters):
    ids = top_most_similar[clt]
    most_similar_titles = '\n'.join(data.iloc[ids]['title'])
    print(f'Cluster {clt}:\n{most_similar_titles}')

In [ ]:
import numpy as np

# следует реализовать данный вид инициализации
# без нее тесты скорее всего не пройдут
def k_plus_plus(X: np.ndarray, k: int, random_state: int = 27) -> np.ndarray:
    """Инициализация центроидов алгоритмом k-means++.

    :param X: исходная выборка
    :param k: количество кластеров
    :return: набор центроидов в одном np.array
    """
    pass

class KMeansCustom:
    def __init__(self, n_clusters=8, tol=0.0001, max_iter=300, random_state=None):
        self.n_clusters = n_clusters
        self.tol = tol
        self.max_iter = max_iter
        self.random_state = random_state

    def fit(self, X):
        np.random.seed(self.random_state)

        n_samples = X.shape[0]
        n_features = X.shape[1]

        # инициализируем центры кластеров
        # centers.shape = (n_clusters, n_features)
        centers = ...

        for n_iter in range(self.max_iter):
            # считаем расстояние от точек из X до центроидов
            distances = ...
            # определяем метки как индекс ближайшего для каждой точки центроида  
            labels = ...

            old_centers = centers.copy()
            for c in range(self.n_clusters):
                # пересчитываем центроид 
                # новый центроид есть среднее точек X с меткой рассматриваемого центроида
                centers[c, :] = ...

            # записываем условие сходимости
            # норма Фробениуса разности центров кластеров двух последовательных итераций < tol
            if ... :
                break

        # cчитаем инерцию
        # сумма квадратов расстояний от точек до их ближайших центров кластеров
        inertia = ...

        self.cluster_centers_ = centers
        self.labels_ = labels
        self.inertia_ = inertia
        self.n_iter_ = n_iter
        return self


    def predict(self, X):
        # определяем метку для каждого элемента X на основании обученных центров кластеров 
        distances = ...
        labels = ...
        return labels

    def fit_predict(self, X):
        return self.fit(X).labels_


kmeans = KMeansCustom(n_clusters=19)
kmeans.fit(w2v_embeddings)

top = 5
clusters = kmeans.labels_
dist_to_centers = euclidean_distances(kmeans.cluster_centers_, w2v_embeddings)
top_most_similar = np.argsort(dist_to_centers)[:, :top]

for clt in np.unique(clusters):
    ids = top_most_similar[clt]
    most_similar_titles = '\n'.join(data.iloc[ids]['title'])
    print(f'Cluster {clt}:\n{most_similar_titles}')

* Визуализация кластеров в 2D с помощью TSNE — широко распространённый метод визуализации данных высокой размерности. В случае очень большой размерности признакового пространства имеет смысл предварительно уменьшить размерность, например, с помощью PCA, для ускорения вычислений и устранения возможного шума.

In [ ]:
# для каждого кластера сэмплируем не более 20 точек для ускорения TSNE и наглядности картинки
sample_i = []
for clt in np.unique(clusters):
    cluster_mask = clusters == clt
    sample_i += random.sample(list(np.where(cluster_mask)[0]), min(20, cluster_mask.sum()))

# берем слайс по sample_i
sample_embeddings = w2v_embeddings[sample_i]
sample_clusters = clusters[sample_i].reshape(-1, 1)
sample_texts = data['title'].values[sample_i]

In [ ]:
# обучаем и применяем TSNE
tsne_model = TSNE(perplexity=50, metric='cosine', init='pca', n_iter=2500, random_state=17)
reduced_embeddings = tsne_model.fit_transform(sample_embeddings)

In [ ]:
not_annotate_thr=0.3
plt.figure(figsize=(18, 18)) 
colors = cm.nipy_spectral(sample_clusters.astype(float) / len(np.unique(sample_clusters)))
for i, (x, y) in enumerate(reduced_embeddings):
    if np.random.rand() > not_annotate_thr:
        plt.scatter(x, y, c=colors[i])
        plt.annotate(sample_texts[i],
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom', 
                     size=8)
plt.show()

In [ ]:
indexes = np.where(sample_texts == 'учебник')
indexes

In [ ]:
sample_texts[indexes]

In [ ]:
reduced_embeddings[indexes]

[Почему так](https://datascience.stackexchange.com/questions/19025/t-sne-why-equal-data-values-are-visually-not-close)

## DBSCAN: краткий обзор

Одним из наиболее часто используемых на практике методов кластеризации при неизвестном количестве кластеров является [DBSCAN](https://ru.wikipedia.org/wiki/DBSCAN)— метод кластеризации, основанный на плотности. 

Метод не применим к данным с большой разницей в плотности кластеров.

Основные преимущества:

1. Не требует априорного знания о необходимом количестве кластеров;
1. Выделяет кластеры произвольной формы;
1. Устойчив к выбросам, выделяет шум.

Подбор параметров

Вся сложность применения метода заключается в подборе двух основных параметров: `min_samples` и `eps`

1. `min_samples` параметр определяет сколько объектов должно содержаться в `eps` окрестности выбранного объекта, чтобы этот объект считалась внутрикластерным. Имеет смысл брать только значения ≥3, на практике часто бывает достаточно ограничить поиски значениями от 3 до 9, но это носит лишь рекомендательный характер и не отрицает наличия оптимальной пары с `eps` вне данного отрезка
1. `eps` параметр определяет размер окрестности. Чтобы больше определиться с областью поиска, для начала нужно выбрать функцию расстояния, так как для ненормированных векторов значения `euclidean_distances` и `cosine_distances` скорее всего лежат в разных диапазонах

Далее можно воспользоваться двумя методами подбора: 
* Метод колена: для выбранного min_samples определяется подходящий eps

*Идея*: для каждого объекта выборки считается расстояние до n-го соседа, где n выбирается равным `min_samples` . После этого расстояния сортируются и отрисовываются. В качестве `eps` берётся значение расстояния в точке с максимальной кривизной, предполагая, что точки с сильно растущим расстоянием до n-го соседа являются выбросами.

Для поиска оптимальной точки можно воспользоваться библиотекой [kneed](https://kneed.readthedocs.io/en/stable/index.html) и ее классом [KneeLocator](https://kneed.readthedocs.io/en/stable/api.html#kneelocator). Суть его заключается в построении интерполирующей функции по заданным точкам (часто из класса полиномиальных функций) и выбора точки (среди заданных) с максимальным значением кривизны полученного интерполянта. В общепринятой формулировке метода колена значения расстояний располагаются в порядке возрастания, следовательно, имеем выпуклую кривую `curve='convex'` с порядком обхода по возрастанию `direction='increasing'`, параметр `interp_method='interp1d'` or `'polynomial'` отвечает методу интерполяции точек.

In [ ]:
!pip install kneed

In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
w2v_embeddings_sample = w2v_embeddings[::5]

In [ ]:
distances = pairwise_distances(w2v_embeddings_sample)
distances.shape

In [ ]:
n = 6

In [ ]:
sorted_distances = np.sort(distances, axis=1)
sorted_nth_distanses = np.sort(sorted_distances[:, n - 1])

In [ ]:
from kneed import KneeLocator
knee = KneeLocator(np.arange(len(sorted_distances)), 
                   sorted_nth_distanses, 
                   curve='convex', 
                   direction='increasing', 
                   interp_method='polynomial')

fig = plt.figure(figsize=(5, 5))
knee.plot_knee()
plt.xlabel("Points")
plt.ylabel("Distance")
print(sorted_nth_distanses[knee.knee])

В качестве параметров для `DBSCAN` выбираются значения `eps=0.84`, `min_samples=7`, `metric='euclidean'`.

На практике бывает полезно перебрать `eps` в небольшой окрестности найденного оптимального значения из-за зависимости оптимума от выбранного метода и степени интерполяции, попробовать разную интерполяцию и разные значения для n.

* Метод силуэта: 

*Идея*: перебирая по сетке значения для `min_samples` и `eps` , выбрать те, которые приводят к наибольшему значению [силуэт-метрики](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html).

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

for eps in np.linspace(0.05, 0.5, 10):
    for ms in [5, 7, 9]:
        clustering = DBSCAN(eps=eps, min_samples=ms, metric='euclidean', n_jobs=-1)
        clusters = clustering.fit_predict(w2v_embeddings_sample)
        score = silhouette_score(w2v_embeddings_sample, clusters)
        print(f'eps = {eps:0.3f}, ms = {ms}: {score:0.4f}')

В домашнем задании будет два пункта:

* Реализовать KMeans
* Выполнить кластеризацию и разбиение заданной категории на виды товаров